## Bypassing device constraints

It is possible that the quantum device constraints prevent us from solving instances. In this advanced tutorial, we show how to:

- provide a custom device in the `BackendConfig` with less tight constraints.
- use a more advanced embedder than the default one. 

Let us first load a graph example. For this case, we expect the MIS to be of size 12.

In [ ]:
# Ignore warnings for this tutorial.
import logging
import os
import sys

logger = logging.getLogger()
logger.disabled = True

sys.stderr = open(os.devnull, 'w')

In [ ]:
import networkx as nx

# Create a new networkx graph instance to be populated with DIMACS data.
graph = nx.Graph()


with open("./datasets/dimacs/a265032_1tc.16.txt", "r") as f:
    for line in f:
        if line.startswith("c"):  # Comment line in DIMACS file.
            continue
        elif line.startswith("p"):  # Problem definition, i.e. # nodes and edges.
            _, _, num_nodes, num_edges = line.strip().split()
            # Preset graph node labels as there might be isolated ones.
            graph.add_nodes_from(range(1, int(num_nodes) + 1))

        elif line.startswith("e"):
            _, node1, node2 = line.strip().split()
            graph.add_edge(int(node1), int(node2))

# Let's check what the graph looks like.
print(graph)


## Solving using a VirtualDevice

### 

When trying to run directly this instance using a quantum approach with the default `BackendConfig`, the embedding will fail as the default device has constraints hard to fit with the `DefaultEmbedder`. Here the embedding will fail due to not respecting the `max_radial_distance`. Let us allievate this by using a `VirtualDevice`. For more details, refer to the [`pulser` tutorial on how to define custom devices](https://pulser.readthedocs.io/en/stable/tutorials/virtual_devices.html).

In [ ]:
from mis import BackendConfig, BackendType, SolverConfig, MISSolver, MISInstance
from pulser.devices import AnalogDevice
from dataclasses import replace

instance = MISInstance(graph)

VirtualAnalog = replace(AnalogDevice.to_virtual(), max_radial_distance=900)

# deactivate preprocessor to run on quantum, as preprocessor will skip running on device
solver_config = SolverConfig(
    # Use the EMU_MPS backend with the virtual device.
    backend = BackendConfig(
        backend = BackendType.EMU_MPS,
        device=VirtualAnalog,
    ),
    # Perform up to 10 quantum measures.
    max_iterations=10,
    preprocessor=None,
    
)

# Run the solver
solver = MISSolver(instance, solver_config)
solutions = solver.solve()

# Display results
print("MIS solution:", solutions[0].nodes)
print("Solution frequency:", solutions[0].frequency)
print("Solution size:", solutions[0].size)
solutions[0].draw()

Also can be run on a remote emulator:

In [ ]:
# Replace with your username, project id and password on the Pasqal Cloud.
USERNAME="username"
PROJECT_ID="123"
PASSWORD=None

if PASSWORD is not None:
    # deactivate preprocessor to run on quantum, as preprocessor will skip running on device
    config = SolverConfig(
        backend = BackendConfig(
            backend=BackendType.REMOTE_EMUMPS,
            device=VirtualAnalog,
            username=USERNAME,
            project_id=PROJECT_ID,
            password=PASSWORD
        ),
        max_iterations=10,
        preprocessor=None,
    )

    # Run the solver
    solver = MISSolver(instance, config)
    solutions = solver.solve()

    # Display results
    print("MIS solution:", solutions[0].nodes)
    print("Solution frequency:", solutions[0].frequency)
    solutions[0].draw()

## Solving with OptimizeEmbedder

`OptimizeEmbedder` extends the default embedder by applying constrained optimization to map the instance given the device constraints.
The optimization may fail though to find a suitable embedding.

In [ ]:
from mis.pipeline.embedder import OptimizedEmbedder

solver_config = SolverConfig(
    # Use the EMU_MPS backend.
    backend = BackendConfig(
        backend = BackendType.EMU_MPS,
    ),
    # Perform up to 10 quantum measures.
    max_iterations=10,
    preprocessor=None,
    embedder=OptimizedEmbedder(),
    
)

# Run the solver
solver = MISSolver(instance, solver_config)
solutions = solver.solve()

# Display results
print("MIS solution:", solutions[0].nodes)
print("Solution frequency:", solutions[0].frequency)
print("Solution size:", solutions[0].size)
solutions[0].draw()